In [4]:
!pip install torch
import shutil
import apiquery
import pandas as pd
import sys
import seaborn as sns
import os
import numpy as np
import random
import torch
import pickle
import gc
DATA_PATH = '../01.Data'
shutil.copy("apiquery_pyc.py", "apiquery.pyc")

module_path = "../src"
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils.training import *
from utils.encoding import *
from utils.utils import *
from utils.fetch import *
from dataset.dataset import BNPParibasText
from models.models import Roberta_Model
from utils.EarlyStopping import EarlyStopping
from utils.LoopFunctions import train_fn,valid_fn
from utils.prediction import get_prediction,get_embedding
from utils.prediction import predict
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
import math
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import math
import time
import lightgbm as lgbm
import matplotlib.pyplot as plt
import torch.nn as nn
import config_ad
import transformers

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [7]:
df_test  = pd.read_csv(os.path.join(DATA_PATH,'test_embeddings.csv'))
columns_modeling = columns_modeling = ['additives_n','ingredients_from_palm_oil_n',
                    'ingredients_that_may_be_from_palm_oil_n',
                    'states_en_brands','states_en_categories','states_en_characteristics','states_en_expiration date',
                    'states_en_general_complete','states_en_ingredients','pnns_groups_1','pnns_groups_2',
                    'states_en_packaging','states_en_packaging-code-','states_en_photo_upload',
                    'states_en_photo_validate','states_en_product name','states_en_quantity','diff_t'] + [i for i in df_test.columns if 'datetime' in i] + [i for i in df_test.columns.to_list() if 'emb' in i]
columns_label = df_test[columns_modeling].select_dtypes(include=['object']).columns.to_list()
print(columns_label)

['states_en_brands', 'states_en_categories', 'states_en_characteristics', 'states_en_expiration date', 'states_en_general_complete', 'states_en_ingredients', 'pnns_groups_1', 'pnns_groups_2', 'states_en_packaging', 'states_en_packaging-code-', 'states_en_photo_upload', 'states_en_photo_validate', 'states_en_product name', 'states_en_quantity', 'emb_codes', 'emb_codes_tags']


In [8]:
a_file = open("../03.Models/General/label_encoding.pkl", "rb")
dict_le = pickle.load(a_file)
df_test = apply_label_encoder(df_test,dict_le,drop_original = True, missing_new_cat = True)
del dict_le,a_file

/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.0 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Mode: Missing as new category
Applying Label Encoding:  label_states_en_brands
Applying Label Encoding:  label_states_en_categories
Applying Label Encoding:  label_states_en_characteristics
Applying Label Encoding:  label_states_en_expiration date
Applying Label Encoding:  label_states_en_general_complete
Applying Label Encoding:  label_states_en_ingredients
Applying Label Encoding:  label_pnns_groups_1
Applying Label Encoding:  label_pnns_groups_2
Applying Label Encoding:  label_states_en_packaging
Applying Label Encoding:  label_states_en_packaging-code-
Applying Label Encoding:  label_states_en_photo_upload
Applying Label Encoding:  label_states_en_photo_validate
Applying Label Encoding:  label_states_en_product name
Applying Label Encoding:  label_states_en_quantity


In [9]:
# Saving Models
models = []
for fold in range(0,5):
    filename = f'../03.Models/Lightgbm/lgbm_fold_best_1_{fold}.pkl'
    print(filename)
    model = pickle.load(open(filename, 'rb'))
    models.append(model)

../03.Models/Lightgbm/lgbm_fold_best_1_0.pkl
../03.Models/Lightgbm/lgbm_fold_best_1_1.pkl
../03.Models/Lightgbm/lgbm_fold_best_1_2.pkl
../03.Models/Lightgbm/lgbm_fold_best_1_3.pkl
../03.Models/Lightgbm/lgbm_fold_best_1_4.pkl


In [10]:
with open('../03.Models/General/feature_list.txt') as f:
    features = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
features = [x.strip() for x in features] 

In [11]:
df_test = predict(df_test,models,features)

fin_predict
fin_predict
fin_predict
fin_predict
fin_predict


In [12]:
y_submission = pd.read_csv(os.path.join(DATA_PATH,'y_test_submission_example.tsv'), index_col='Index', encoding='utf-8', sep='\t')
y_submission['target'] = df_test['target'].values
y_submission .to_csv(os.path.join(DATA_PATH,'y_submission_final.csv'),index = False)

In [11]:
#Enviar los resultados
apiquery.submit_api(y_submission,
       competition_name='food',
        subname='test_v12', # Pueden cambiar esto sin problemas, poner el nombre que quieran.
        holdout_key='None',
        update_ldb=True,
        username="Insight ML - DD" # Poner el nombre de su equipo como un string. 
                                  # El mejor de los resultados dentro de sus envios es el que aparecera en la tabla de posiciones.
)